In [ ]:
import os
import sys
import matplotlib.pyplot as plt
from hydromt.log import setuplog
from hydromt_sfincs import SfincsModel

In [ ]:
# Initialize SfincsModel with the artifact data catalog which contains data for North Italy
sf = SfincsModel(data_libs=["artifact_data"], root="tmp_example", mode="w+", logger=setuplog('', log_level=20))

In [ ]:
sf.setup_grid(
    x0 = 268650,
    y0 = 5018550,
    dx = 150.0,
    dy = 150.0,
    nmax = 272,
    mmax = 425,
    rotation = 0,
    epsg = 32633
)

In [ ]:
sf.setup_config(
    tref = '20100201 000000',
    tstart = '20100201 000000',
    tstop = '20100210 000000',
)

In [ ]:
# 2d precip
sf.setup_precip_forcing_from_grid(
    precip_fn = 'era5_hourly',
    aggregate=True,
)
sf.write_forcing()

In [ ]:
# 1d uniform precip
sf.setup_precip_forcing_from_grid(
    precip_fn = 'era5_hourly',
    aggregate=True,
)



In [ ]:
sf.forcing['precip'].to_pandas().to_csv('precip.csv')
sf.forcing.pop('precip', None)	# reset
sf.setup_precip_forcing(
    timeseries_fn = 'precip.csv',
)
sf.plot_forcing()

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from hydromt_sfincs import utils

df = utils.read_timeseries("sfincs_coastal//sfincs.bzs", tref=sf.config['tref'])
gdf = utils.read_xy("sfincs_coastal//sfincs.bnd", crs=sf.crs)

sf.forcing.pop('bzs', None)	# reset
sf.forcing.pop('precip', None)	# reset
# add only locations
sf.create_waterlevel_forcing(
    gdf_locs=gdf,
)
# add timeseries to existing locations
sf.create_waterlevel_forcing(
    df_ts=df,
    merge=True,
)
sf.plot_forcing()

In [ ]:
# merge (overwrite) existing timeseries with different time resoltiuon
# and add offset
sf.create_waterlevel_forcing(
    df_ts=df.iloc[::5,[0]],
    gdf_locs=gdf.iloc[[0]],
    da_offset=sf.data_catalog.get_rasterdataset('dtu10mdt'),
    merge=True,
)
sf.plot_forcing()

In [ ]:
# update timeseries from csv
df.to_csv("waterlevel.csv")
sf.setup_waterlevel_forcing(
    timeseries_fn='waterlevel.csv',
    merge=True,
)
sf.plot_forcing()


In [ ]:
# overwrite forcing from geodataset (netcdf file)
sf.setup_waterlevel_forcing(
    geodataset_fn='gtsmv3_eu_era5',
    offset_fn='dtu10mdt',
    merge=False,
)
sf.plot_forcing()

In [ ]:
sf.write_forcing()
sf.write_config()

In [ ]:
# note that index number cannot be saved in ascii timeseries format 
# and are stored as attributes of the geojson file
sf1 = SfincsModel(sf.root, mode='r') # read mode
sf1.read_forcing()
sf1.plot_forcing()